In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

In [ ]:
# Lock seed
tf.set_random_seed(1)
np.random.seed(1)

In [ ]:
# Load datasets
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('.', one_hot=True)

In [ ]:
plt.figure(figsize=(10, 2))
for i in range(5):
    plt.subplot(151 + i)
    plt.title('num: {}'.format(np.argmax(mnist.train.labels[i])))
    plt.axis('off')
    plt.imshow((mnist.train.images[i].reshape(28, 28) * 255))
plt.show()

In [ ]:
num_label_list = [np.sum(np.argmax(mnist.train.labels, 1) == num) for num in range(10)]
num_labels = ['num: {}, count: {}'.format(num, num_label_list[num]) for num in range(10)]
plt.figure(figsize=(10, 8))
plt.title('Train datasets distribution')
plt.pie(num_label_list, explode=np.ones((10, )) * 0.05, labels=num_labels, shadow=True)
plt.show()

In [ ]:
# Config Model.

learning_rate = 1.
batch_size = 128
n_epochs = 25

input_dim = 784
n_classes = 10
dropout_rate = 0.75 # keep prob

In [ ]:
# Init placeholds.

X = tf.placeholder('float32', shape=(None, input_dim))
Y = tf.placeholder('float32', shape=(None, n_classes))
keep_prob = tf.placeholder('float32')

In [ ]:
# Define functions.

def conv2d(X, W, b):
    conv = tf.nn.conv2d(X, W, strides=(1, 1, 1, 1), padding='VALID')
    return tf.nn.bias_add(conv, b)

def maxpool2d(X, k=2):
    pool = tf.nn.max_pool(X, ksize=(1, k, k, 1), strides=(1, k, k, 1), padding='SAME')
    return tf.nn.relu(pool)

In [ ]:
# Define CNN

def CNN(X, Weights, biases, dropout):
    X = tf.reshape(X, shape=(-1, 28, 28, 1))
    
    conv1 = conv2d(X, Weights['W1'], biases['b1'])
    conv1 = maxpool2d(conv1)
    
    conv2 = conv2d(conv1, Weights['W2'], biases['b2'])
    conv2 = maxpool2d(conv2)
    
    fc1 = tf.reshape(conv2, shape=(-1, Weights['W3'].get_shape().as_list()[0]))
    fc1 = tf.add(tf.matmul(fc1, Weights['W3']), biases['b3'])
    fc1 = tf.nn.relu(fc1)
    
    fc1 = tf.nn.dropout(fc1, dropout)
    
    out = tf.add(tf.matmul(fc1, Weights['W4']), biases['b4'])
    
    return out

In [ ]:
# Init Weights, biases
Weights = {
    'W1': tf.Variable(tf.random_normal((5, 5, 1, 10), stddev=0.1)),
    'W2': tf.Variable(tf.random_normal((5, 5, 10, 20), stddev=0.1)),
    'W3': tf.Variable(tf.random_normal((320, 128), stddev=0.1)),
    'W4': tf.Variable(tf.random_normal((128, n_classes), stddev=0.1))
}

biases = {
    'b1': tf.Variable(tf.constant(0.1, dtype='float32', shape=(10, ))),
    'b2': tf.Variable(tf.constant(0.1, dtype='float32', shape=(20, ))),
    'b3': tf.Variable(tf.constant(0.1, dtype='float32', shape=(128, ))),
    'b4': tf.Variable(tf.constant(0.1, dtype='float32', shape=(10, )))
}

In [ ]:
# Define optimizer
pred = CNN(X, Weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y))
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
# Evaluate model
currect_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(currect_pred, 'float32'))

In [ ]:
train_loss_list = [1]
train_acc_list = [0]
test_loss_list = [1]
test_acc_list = [0]

init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        train_loss, train_acc = 0., 0.
        times = mnist.train.num_examples // batch_size
        for time in range(times):
            batch_X, batch_Y = mnist.train.next_batch(batch_size)
            sess.run(
                optimizer,
                feed_dict={X: batch_X, Y: batch_Y, keep_prob: dropout_rate})
            _loss, _acc = sess.run(
                [cost, accuracy],
                feed_dict={X: batch_X, Y: batch_Y, keep_prob: 1.})
            train_loss += _loss / times
            train_acc += _acc / times
        
        test_loss, test_acc = 0., 0.
        times = mnist.test.num_examples // 100
        for time in range(times):
            batch_X, batch_Y = mnist.test.next_batch(100)
            _loss, _acc = sess.run(
                [cost, accuracy],
                feed_dict={X: batch_X, Y: batch_Y, keep_prob: 1.})
            test_loss += _loss / times
            test_acc += _acc / times

        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        test_loss_list.append(test_loss)
        test_acc_list.append(test_acc)
        print('Epoch:', epoch + 1)
        print('train loss = {}, train accracy = {}'.format(train_loss, train_acc))
        print('test loss = {}, test accracy = {}'.format(test_loss, test_acc))
        print('-' * 64)
    plt.figure(figsize=(10, 8))
    plt.xlabel('Epochs')
    plt.ylabel('Rate')
    plt.plot(test_loss_list, '-b', label='test loss', linewidth=2)
    plt.plot(test_acc_list, '-y', label='test accuracy', linewidth=2)
    plt.plot(train_loss_list, '--r', label='train loss', linewidth=2)
    plt.plot(train_acc_list, '--g', label='train accuracy', linewidth=2)
    plt.legend()
    plt.show()